In [6]:
import json
import numpy as np
from tqdm import tqdm
import pandas as pd

from dataclasses import dataclass, asdict

import os
import requests
import uuid
from urllib.parse import urlparse
from PIL import Image

def download_image(url_or_path, save_dir="save_dir"):
    """
    Checks if the given link is a local image path or a URL.
    If it's a URL, it downloads the image and saves it with a unique name.

    Args:
        url_or_path (str): The file path or URL of the image.
        save_dir (str): The directory to save the images. Defaults to "save_dir".

    Returns:
        str or None: The path to the saved file if the image was downloaded,
                     or the original path if it was a local file,
                     or None in case of an error.
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Folder '{save_dir}' created.")

    # Check if it's a URL
    parsed_url = urlparse(url_or_path)
    if parsed_url.scheme in ('http', 'https', 'ftp', 'ftps'):
        try:
            response = requests.get(url_or_path, stream=True)
            response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)

            # Generate a unique file name
            file_extension = os.path.splitext(parsed_url.path)[1]
            if not file_extension:
                # Attempt to determine extension from Content-Type if not in URL
                content_type = response.headers.get('Content-Type')
                if 'jpeg' in content_type:
                    file_extension = '.jpg'
                elif 'png' in content_type:
                    file_extension = '.png'
                elif 'gif' in content_type:
                    file_extension = '.gif'
                else:
                    file_extension = '.bin' # Fallback

            unique_filename = str(uuid.uuid4()) + file_extension
            save_path = os.path.join(save_dir, unique_filename)

            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Image successfully downloaded and saved: {save_path}")
            return save_path
        except requests.exceptions.RequestException as e:
            print(f"Error downloading image from URL '{url_or_path}': {e}")
            return None
    elif os.path.exists(url_or_path):
        print(f"Link '{url_or_path}' is a local file.")
        return url_or_path
    else:
        print(f"Link '{url_or_path}' is neither a valid URL nor a local file.")
        return None
    
def highlight_predictions(row):
    """
    Applies CSS styles to a row of a DataFrame to highlight model predictions.
    
    This function is intended for use with `DataFrame.style.apply(axis=1)`.

    - Green: Correct prediction (cell value matches the 'label' column).
    - Salmon: Incorrect prediction.
    - Light Gray: No prediction was made (cell value is None).
    - No Style: Applied to non-model columns (e.g., 'image_id').
    
    Args:
        row (pd.Series): A single row of the DataFrame.
        
    Returns:
        list: A list of CSS style strings, one for each cell in the row.
    """
    # Get the ground truth label for the current row
    label = row['label']
    
    # Initialize a list to hold the style for each cell in the row
    styles = []
    
    # Iterate over the index (column names) of the row
    for col_name in row.index:
        # Check if the column is a model prediction column
        if col_name.startswith('model_'):
            value = row[col_name]
            if value is None:
                styles.append('background-color: lightgray')
            elif value == label:
                styles.append('background-color: lightgreen')
            else:
                styles.append('background-color: salmon')
        else:
            # No style for non-model columns ('image_id', 'annotation_id', 'label')
            styles.append('')
            
    return styles

def run_inference(crop_rgb_np: np.ndarray, models: dict, max_k: int) -> dict:
    """
    Runs inference on all models for a single crop and formats the result.

    Args:
        crop_rgb_np: The image crop as a NumPy array.
        models: A dictionary of models to run inference with.
        max_k: The maximum number of predictions to save.

    Returns:
        A dictionary containing predictions from all models.
    """
    model_predictions = {}
    for model_name, model_obj in models.items():
        # Get predictions from the model
        if model_name == 'model_v92':
            input_np_img = crop_rgb_np[:, :, ::-1] # model versiob 92 work on bgr image
        else:
            input_np_img = crop_rgb_np
            
        prediction_group = model_obj.inference_numpy(input_np_img)
        output_labels = [asdict(pred)['name'] for pred in prediction_group]

        # Save the top-K predictions
        for k in range(max_k):
            key = f"{model_name}_@{k+1}"
            try:
                model_predictions[key] = output_labels[k]
            except IndexError:
                # If there are fewer predictions than k, fill with None
                model_predictions[key] = None

    return model_predictions


def process_and_save_results(images_dict: dict, models: dict, max_k: int, output_path: str):
    """
    Main function to process images, run inference, and save the results.
    """
    all_results = []

    print(f"Processing images with {len(models)} models...")

    for image_id, image_data in images_dict.items():
        image_path = download_image(image_data['image_url'])
        try:
            img_pil_rgb = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Could not open image {image_id} from {image_path}. Error: {e}")
            continue

        for annotation in image_data['annotations']:
            # Prepare the data for a single row in the final file
            record = {
                'image_id': image_id,
                'annotation_id': annotation['id'],
                'label': annotation['label']
            }

            # Crop the image based on the annotation
            x, y, w, h = annotation['bbox']
            xyxy = (x, y, x + w, y + h)
            crop_rgb_pil = img_pil_rgb.crop(xyxy)
            crop_rgb_np = np.array(crop_rgb_pil)
            
            # Get predictions from all models
            predictions = run_inference(crop_rgb_np, models, max_k)
            
            # Add the predictions to the record
            record.update(predictions)

            # Append the complete row to the main list
            all_results.append(record)
    
    if not all_results:
        print("No results to save.")
        return

    print("Creating DataFrame and saving to Excel...")
    
    # Creating a DataFrame from a list of dictionaries is efficient
    df = pd.DataFrame(all_results)
    
    # Apply styling and save
    styled_df = df.style.apply(highlight_predictions, axis=1)
    styled_df.to_excel(output_path, engine="openpyxl", index=False)
    
    print(f"Successfully saved results to {output_path}")
    

In [2]:
from models.class_v93.inference import EmbeddingClassifier as EmbeddingClassifierV93

classifier_configv93 = {
        'model':{
            'path': 'models/class_v93/model.ckpt',
            'device': 'cpu'
        },
        'dataset':{
            'path': 'models/class_v93/database.pt'
        },
        'log_level': 'INFO' 
    }
    
model_classifier_v93 = EmbeddingClassifierV93(classifier_configv93)

from models.class_v92.inference import EmbeddingClassifier as EmbeddingClassifierV92

classifier_configv92 = {
        'model':{
            'path': 'models/class_v92/model.ckpt',
            'device': 'cpu'
        },
        'dataset':{
            'path': 'models/class_v92/database.pt'
        },
        'log_level': 'INFO'
    }
    
model_classifier_v92 = EmbeddingClassifierV92(classifier_configv92)

[2025-06-24 12:38:49] [INFO] - Dataset loaded from models/class_v93/database.pt
[2025-06-24 12:38:50] [INFO] - Initializing EmbeddingClassifier...


Loading ViT backbone: beitv2_base_patch16_224.in1k_ft_in22k_in1k


[2025-06-24 12:38:51] [INFO] - Torch model loaded from models/class_v93/model.ckpt
[2025-06-24 12:38:51] [INFO] - EmbeddingClassifier initialized successfully.


StableEmbeddingModel initialized with ViT backbone: beitv2_base_patch16_224.in1k_ft_in22k_in1k
  Embedding Dim: 512, Num Classes: 639
  ArcFace s: 64.0, m: 0.5
  Backbone out features (ViT embed_dim): 768
  BN in embedding: False, Dropout in embedding: 0.11


[2025-06-24 12:38:51] [INFO] - Dataset loaded from models/class_v92/database.pt
[2025-06-24 12:38:51] [INFO] - Initializing EmbeddingClassifier...
/home/fishial/miniconda3/envs/nemo_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fishial/miniconda3/envs/nemo_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[2025-06-24 12:38:52] [INFO] - Torch model loaded from models/class_v92/model.ckpt
[2025-06-24 12:38:52] [INFO] - EmbeddingClassifier initialized successfully.


In [3]:
# ================== CONFIG ==================
# 1. Set the maximum number of top predictions to save
MAX_TOP_K = 3

# 2. Define all the models you want to use.
#    The key is the name that will be used in the column headers.
#    The value is your model object, which must have an `inference_numpy` method.
MODELS_TO_RUN = {
    "model_v92": model_classifier_v92,
    "model_v93": model_classifier_v93,
    # Add other models here following the same pattern
    # "another_model": load_another_model(),
}

# 3. Name of the output Excel file
OUTPUT_FILENAME = "v9_test_dataset.xlsx"
# ============================================

In [9]:
import json
import numpy as np
from collections import defaultdict

# Define the path to the COCO JSON file
coco_file_path = '/home/fishial/Fishial/dataset/EXPORT_V_0_9/Fishial_Export_Mar_25_2025_02_47_Prod_Export_Test_Images_for_testing.json'

# Load the COCO data from the file
try:
    with open(coco_file_path, 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print(f"Error: The file '{coco_file_path}' was not found.")
    exit()
except json.JSONDecodeError:
    print(f"Error: The file '{coco_file_path}' is not a valid JSON file.")
    exit()

id_to_label = {
    category['id']: category['supercategory']
    for category in data.get('categories', [])
    if category.get('name') == 'General body shape'
}

# Initialize the images dictionary using a dictionary comprehension.
# Using defaultdict to simplify appending annotations.
images_dict = {
    image['id']: {
        'image_url': image['coco_url'],
        'annotations': []
    }
    for image in data.get('images', [])
}

# Process annotations and add them to the corresponding images
for annotation in data.get('annotations', []):
    category_id = annotation.get('category_id')
    image_id = annotation.get('image_id')

    # Check if the annotation is valid and relevant
    if 'bbox' in annotation and image_id in images_dict and category_id in id_to_label:
        images_dict[image_id]['annotations'].append({
            'bbox': np.array(annotation['bbox']).astype(np.int32),
            'label': id_to_label[category_id],
            'id': annotation['id']
        })

# The 'images_dict' is now populated with the desired data.
# You can now, for example, print the number of images and annotations found.
print(f"Processed {len(images_dict)} images.")
total_annotations = sum(len(img['annotations']) for img in images_dict.values())
print(f"Found {total_annotations} relevant annotations.")

Processed 11847 images.
Found 12045 relevant annotations.


In [ ]:
# Start the main process
process_and_save_results(
    images_dict=images_dict,
    models=MODELS_TO_RUN,
    max_k=MAX_TOP_K,
    output_path=OUTPUT_FILENAME
)

Processing images with 2 models...


[2025-06-24 15:00:19] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:19] [INFO] - Completed in 0.00 seconds


Image successfully downloaded and saved: save_dir/2e8e45d3-d9ef-4bcf-8dbf-8976c7e8f518.jpg


[2025-06-24 15:00:19] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:19] [INFO] - Completed in 0.01 seconds
[2025-06-24 15:00:19] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:19] [INFO] - Completed in 0.01 seconds
[2025-06-24 15:00:20] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:20] [INFO] - Completed in 0.03 seconds
[2025-06-24 15:00:20] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:20] [INFO] - Completed in 0.01 seconds
[2025-06-24 15:00:20] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:20] [INFO] - Completed in 0.02 seconds
[2025-06-24 15:00:20] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:20] [INFO] - Completed in 0.02 seconds
[2025-06-24 15:00:20] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:20] [INFO] - Completed in 0.01 seconds
[2025-06-24 15:00:22] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:00:22] [INFO] - Completed in 0.02 seconds
[2025-06-24 15:00:22] [I

Image successfully downloaded and saved: save_dir/808b0c85-9111-4ffc-85da-cc7f08cb786e.jpg


...

[2025-06-24 15:01:22] [INFO] - Completed in 0.01 seconds
[2025-06-24 15:01:22] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:01:22] [INFO] - Completed in 0.03 seconds


Image successfully downloaded and saved: save_dir/3f7ac1a2-cb20-4e3e-91b7-d6d4923dc09b.jpg


[2025-06-24 15:01:22] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:01:22] [INFO] - Completed in 0.02 seconds
[2025-06-24 15:01:23] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:01:23] [INFO] - Completed in 0.01 seconds


Image successfully downloaded and saved: save_dir/c37e707a-692e-4137-aa1b-ba3eceb6f103.jpg


[2025-06-24 15:01:23] [INFO] - Starting search over 1 embeddings
[2025-06-24 15:01:23] [INFO] - Completed in 0.02 seconds
[2025-06-24 15:01:23] [INFO] - Starting search over 1 embeddings
